# 特征工程

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv('../data/source/usedcar_train.csv', sep=" ")
test = pd.read_csv('../data/source/usedcar_test.csv', sep=" ")

train['price'] = np.log1p(train['price'])
print('Train data shape:',train.shape)
print('TestA data shape:',test.shape)

#合并数据集
data = pd.concat([train, test],ignore_index=True)

Train data shape: (150000, 31)
TestA data shape: (50000, 30)


/opt/anaconda3/envs/csdn/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)

## 1. 特征基本统计

In [4]:
date_cols = ['regDate', 'creatDate']
cate_cols = ['name', 'model', 'brand', 'bodyType', 'fuelType', 'gearbox', 'notRepairedDamage', 'regionCode', 'seller', 'offerType']
num_cols = ['power', 'kilometer'] + ['v_{}'.format(i) for i in range(15)]
cols = date_cols + cate_cols + num_cols

tmp = pd.DataFrame()
tmp['count'] = data[cols].count().values
tmp['missing_rate'] = (data.shape[0] - tmp['count']) / data.shape[0]
tmp['nunique'] = data[cols].nunique().values
tmp['max_value_counts'] = [data[f].value_counts().values[0] for f in cols]
tmp['max_value_counts_prop'] = tmp['max_value_counts'] / data.shape[0]
tmp['max_value_counts_value'] = [data[f].value_counts().index[0] for f in cols]
tmp.index = cols
tmp

## 2. 处理power

In [5]:
#处理异常值
def smooth_cols(group,cols = ['power'],out_value = 600):
    for col in cols:
        yes_no = (group[col]<out_value).astype('int')
        new = yes_no * group[col]
        group[col] = new.replace(0,group[col].median())
    return group

data = data.groupby('brand').apply(smooth_cols,cols = ['power'],out_value = 600)
data.index = range(len(data))


## 3.   处理时间

In [6]:
from tqdm import tqdm

#提取日期信息
date_cols = ['regDate', 'creatDate']

# data.loc[data['regDate'] == 20070009]
# 使用时间：data['creatDate'] - data['regDate']，反应汽车使用时间，一般来说价格与使用时间成反比
# 数据里有时间出错的格式，需要加errors='coerce'
data['used_time_days'] = (pd.to_datetime(data['creatDate'], format='%Y%m%d', errors='coerce') - 
                            pd.to_datetime(data['regDate'], format='%Y%m%d', errors='coerce')).dt.days
data['used_time_month'] = round(data['used_time_days'] / 30, 3)
data['used_time_year'] = round(data['used_time_days'] / 365, 3)

def date_proc(x):
    m = int(x[4:6])
    if m == 0:
        m = 1
    return x[:4] + '-' + str(m) + '-' + x[6:]

for col in tqdm(date_cols):
    data[col] = pd.to_datetime(data[col].astype('str').apply(date_proc))
    data[col + '_year'] = data[col].dt.year
    data[col + '_month'] = data[col].dt.month
    data[col + '_day'] = data[col].dt.day
    data[col + '_dayofweek'] = data[col].dt.dayofweek
    
# 增加新特征，成交日期是否是周末
data['is_weekend'] = data['creatDate_dayofweek'].apply(lambda x: 1 if x in(5,6) else 0)

# 使用年限折旧
def depreciation_year(year):
    if year <= 3:
        return 1 - year * 0.15
    elif year > 3 and  year <= 7:
        return 0.55 - (year-3) * 0.1
    elif year > 7 and  year <= 10:
        return 0.25 - (year-7) * 0.05
    else:
        return 0

data['depreciation_year'] = data['used_time_year'].apply(lambda x: depreciation_year(x))

100%|██████████| 2/2 [00:01<00:00,  1.87it/s]


In [7]:
# 将2及之后fuelType的都归为2
# Train_data.loc[Train_data['fuelType'] >= 2,'fuelType'] = 2 #用这个代码会直接把空值也变成2
data.loc[data['fuelType'] == 3,'fuelType'] = 2 
data.loc[data['fuelType'] == 4,'fuelType'] = 2 
data.loc[data['fuelType'] == 5,'fuelType'] = 2 
data.loc[data['fuelType'] == 6,'fuelType'] = 2 
data['fuelType'].value_counts()

data['notRepairedDamage'].replace('-', '2.0', inplace=True)
print('concat data shape:',data.shape)

# 对类别较少的特征采用one-hot编码
# one_hot_list = ['fuelType','gearbox','notRepairedDamage']
# for col in one_hot_list:
#     one_hot = pd.get_dummies(data[col])
#     one_hot.columns = [col+'_'+str(i) for i in range(len(one_hot.columns))]
#     data = pd.concat([data,one_hot],axis=1)

concat data shape: (200000, 44)


In [8]:
missing = data.isnull().sum()
missing = missing[missing>0]
missing.sort_values(inplace=True)
missing

model                  1
bodyType            5919
gearbox             7891
fuelType           11573
used_time_days     15101
used_time_month    15101
used_time_year     15101
price              50000
dtype: int64

## 4. 处理缺失值

In [9]:
train['notRepairedDamage'].replace('-', 2.0, inplace=True)

features = ['model','bodyType','fuelType','gearbox']
for fe in features:
    data[fe].fillna(data[fe].mode()[0], inplace=True)
    train[fe].fillna(data[fe].mode()[0], inplace=True)
    test[fe].fillna(data[fe].mode()[0], inplace=True)
    
features = ['used_time_days','used_time_month','used_time_year']
for fe in features:
    data[fe].fillna(data[fe].median(), inplace=True)

## 5. 提取城市，年平均里程和独热编码

In [10]:
# 从邮编中提取城市信息，相当于加入了先验知识
data['city'] = data['regionCode'].apply(lambda x : str(x)[:-3])
data['city'].replace('', 0, inplace=True)

# 计算年平均里程， 即kilometer/汽车使用年限
data['kilometer_everyear'] = round(1000 * data['kilometer'] / data['used_time_year'],3)

## 7. 统计特征在数据集里出现次数，代表热门程度

In [11]:
from tqdm import tqdm

# count编码
def count_features(df, feat_cols):
    for feat in tqdm(feat_cols):
        df[feat + '_count'] = df[feat].map(df[feat].value_counts())
    return(df)

feature_list = ['regDate','creatDate','regDate_year','model','brand','regionCode','bodyType',
                'fuelType','gearbox','notRepairedDamage']
data = count_features(data, feature_list)

100%|██████████| 10/10 [00:00<00:00, 72.13it/s]


In [12]:
# 计算某品牌的销售统计量，同学们还可以计算其他特征的统计量
# 这里要以 train 的数据计算统计量
Train_gb = train.groupby("brand")
all_info = {}
for kind, kind_data in Train_gb:
    info = {}
    kind_data = kind_data[kind_data['price'] > 0]
    info['brand_amount'] = len(kind_data)
    info['brand_price_max'] = kind_data.price.max()
    info['brand_price_median'] = kind_data.price.median()
    info['brand_price_min'] = kind_data.price.min()
    info['brand_price_sum'] = kind_data.price.sum()
    info['brand_price_ptp'] = kind_data.price.ptp()
    info['brand_price_std'] = kind_data.price.std()
    info['brand_price_average'] = round(kind_data.price.sum() / (len(kind_data) + 1), 2)
    all_info[kind] = info
brand_fe = pd.DataFrame(all_info).T.reset_index().rename(columns={"index": "brand"})
data = data.merge(brand_fe, how='left', on='brand')

/opt/anaconda3/envs/csdn/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  del sys.path[0]


In [13]:
# 计算某品牌的销售统计量，同学们还可以计算其他特征的统计量
# 这里要以 train 的数据计算统计量
Train_gb = train.groupby("model")
all_info = {}
for kind, kind_data in Train_gb:
    info = {}
    kind_data = kind_data[kind_data['price'] > 0]
    info['model_amount'] = len(kind_data)
    info['model_price_max'] = kind_data.price.max()
    info['model_price_median'] = kind_data.price.median()
    info['model_price_min'] = kind_data.price.min()
    info['model_price_sum'] = kind_data.price.sum()
    info['model_price_ptp'] = kind_data.price.ptp()
    info['model_price_std'] = kind_data.price.std()
    info['model_price_average'] = round(kind_data.price.sum() / (len(kind_data) + 1), 2)
    all_info[kind] = info
model_fe = pd.DataFrame(all_info).T.reset_index().rename(columns={"index": "model"})
data = data.merge(model_fe, how='left', on='model')

/opt/anaconda3/envs/csdn/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  del sys.path[0]


In [14]:
# 构建统计量函数
def Group_Statistic(train,data,feature,Target):
    Train_gb = train.groupby(feature)
    all_info = {}
    for kind, kind_data in Train_gb:
        info = {}
        kind_data = kind_data[kind_data[Target] > 0]
#         info[feature + '_amount'] = len(kind_data)
        info[feature + '_' + Target + '_max'] = kind_data[Target].max()
        info[feature + '_' + Target + '_median'] = kind_data[Target].median()
        info[feature + '_' + Target + '_min'] = kind_data[Target].min()
        info[feature + '_' + Target + '_sum'] = kind_data[Target].sum()
        info[feature + '_' + Target + '_std'] = kind_data[Target].std()
        info[feature + '_' + Target + '_average'] = round(kind_data[Target].sum() / (len(kind_data) + 1), 2)
        all_info[kind] = info
    new_df = pd.DataFrame(all_info).T.reset_index().rename(columns={'index': feature})
    new_df[feature] = new_df[feature].astype(type(data[feature][0]))
    return data.merge(new_df, how='left', on=feature)

In [15]:
data['used_time_bin'] = pd.cut(data['used_time_month'], bins=30, labels=False, include_lowest=True)
train['used_time_month'] = data.iloc[:len(train),:]['used_time_month']
data = Group_Statistic(train,data,'brand','used_time_month')
data = Group_Statistic(train,data,'model','used_time_month')
# data = Group_Statistic(train,data,'bodyType', 'price')

# 构建 model + brand 和 price 的统计量
data['brand'] = data['brand'].map(lambda x:str(x))
data['model'] = data['model'].map(lambda x:str(x))
data['model_brand'] = data['model'].str.cat(data['brand'], sep = '_')
train['model_brand'] = data.iloc[:len(train),:]['model_brand']

data = Group_Statistic(train,data,'model_brand','price')
data = data.drop(['model_brand'], axis=1)

# data['brand'] = data['brand'].astype(np.int64)
# data['model'] = data['model'].astype(np.float64)

## 8. 用数值特征对类别特征做统计刻画

In [16]:
#定义交叉特征统计
def cross_cat_num(df,cat_col,num_col):
    for f1 in tqdm(cat_col):
        g = df.groupby(f1, as_index=False)
        for f2 in tqdm(num_col):
            feat = g[f2].agg({
                '{}_{}_max'.format(f1, f2): 'max', '{}_{}_min'.format(f1, f2): 'min',
                '{}_{}_median'.format(f1, f2): 'median', '{}_{}_mean'.format(f1, f2): 'mean',
                '{}_{}_std'.format(f1, f2): 'std', '{}_{}_mad'.format(f1, f2): 'mad',
            })
            df = df.merge(feat, on=f1, how='left')
    return(df)

# 用数值特征对类别特征做统计刻画，挑了几个跟price相关性最高的匿名特征
cross_cat = ['model', 'brand', 'regionCode']
cross_num = ['v_0', 'v_3', 'v_8', 'v_12']
data = cross_cat_num(data,cross_cat,cross_num)

100%|██████████| 3/3 [00:28<00:00,  9.34s/it]


In [19]:
missing = data.isnull().sum()
missing = missing[missing>0]
missing.sort_values(inplace=True)
missing

price    50000
dtype: int64

In [18]:
features_std = ['model_price_std','model_v_0_std','model_v_3_std','model_v_8_std', 'model_v_12_std', 'regionCode_v_0_std', 'regionCode_v_3_std',
                'regionCode_v_8_std','regionCode_v_12_std','model_brand_price_std','model_used_time_month_std']

for feat in features_std:
    data[feat].fillna(data[feat].min(), inplace=True)

## 9. 类别特征的二阶交叉

In [20]:
from scipy.stats import entropy

for f_pair in tqdm([['model', 'brand'],['model', 'regionCode'],['brand', 'regionCode']]):
   
    # 共现次数
    data['_'.join(f_pair) + '_count'] = data.groupby(f_pair)['SaleID'].transform('count')
                    
    # nunique、熵
    data = data.merge(data.groupby(f_pair[0], as_index=False)[f_pair[1]].agg({
        '{}_{}_nunique'.format(f_pair[0], f_pair[1]): 'nunique',
        '{}_{}_ent'.format(f_pair[0], f_pair[1]): lambda x: entropy(x.value_counts() / x.shape[0])}), on=f_pair[0], how='left')
    
    data = data.merge(data.groupby(f_pair[1], as_index=False)[f_pair[0]].agg({
        '{}_{}_nunique'.format(f_pair[1], f_pair[0]): 'nunique',
        '{}_{}_ent'.format(f_pair[1], f_pair[0]): lambda x: entropy(x.value_counts() / x.shape[0])}), on=f_pair[1], how='left')

    # 比例偏好
    data['{}_in_{}_prop'.format(f_pair[0], f_pair[1])] = data['_'.join(f_pair) + '_count'] / data[f_pair[1] + '_count']
    data['{}_in_{}_prop'.format(f_pair[1], f_pair[0])] = data['_'.join(f_pair) + '_count'] / data[f_pair[0] + '_count']

100%|██████████| 3/3 [00:33<00:00, 11.24s/it]


## 10. 后验概率

### 1) bodyType, fuelType, gearBox在品牌brand中的概率

In [34]:
# bodyType, fuelType, gearBox字段做独热编码, 并计算相关概率
dummy_feat = ['bodyType', 'fuelType', 'gearbox']
concat = data.drop(['train','price'], axis=1)

for feat in dummy_feat:
    dummies = pd.get_dummies(concat[feat])
    dummies.columns = ['brand_%s_'%feat + '%s'%col for col in dummies.columns]
    dummies['brand'] = concat['brand'].values
    dummies = dummies.groupby('brand').mean()
    dummies.reset_index()
    data = data.merge(dummies, on='brand', how='left')

### 2) bodyType, fuelType, gearBox, model在地区regionCode中的概率

In [35]:
# bodyType, fuelType, gearBox, model字段做独热编码, 并计算相关概率
dummy_feat = ['bodyType', 'fuelType', 'gearbox','model']
concat = data.drop(['train','price'], axis=1)

for feat in dummy_feat:
    dummies = pd.get_dummies(concat[feat])
    dummies.columns = ['regionCode_%s_'%feat + '%s'%col for col in dummies.columns]
    dummies['regionCode'] = concat['regionCode'].values
    dummies = dummies.groupby('regionCode').mean()
    dummies.reset_index()
    data = data.merge(dummies, on='regionCode', how='left')

## 11. 保存处理好的特征

In [22]:
## 选择特征列
numerical_cols = data.columns
print(numerical_cols)

cat_fea = ['offerType','seller']
feature_cols = [col for col in numerical_cols if col not in cat_fea]
feature_cols = [col for col in feature_cols if col not in ['price']]

## 提前特征列，标签列构造训练样本和测试样本
X_data = data.iloc[:len(train),:][feature_cols]
Y_data = train['price']
X_test  = data.iloc[len(train):,:][feature_cols]

#删除已经编码的特征
# drop_list = one_hot_list + feature_list
drop_list = ['creatDate']

X_data = X_data.drop(drop_list,axis=1)
X_test = X_test.drop(drop_list,axis=1)

print(X_data.shape)
print(X_test.shape)

Index(['SaleID', 'bodyType', 'brand', 'creatDate', 'fuelType', 'gearbox',
       'kilometer', 'model', 'name', 'notRepairedDamage',
       ...
       'regionCode_model_ent', 'model_in_regionCode_prop',
       'regionCode_in_model_prop', 'brand_regionCode_count',
       'brand_regionCode_nunique', 'brand_regionCode_ent',
       'regionCode_brand_nunique', 'regionCode_brand_ent',
       'brand_in_regionCode_prop', 'regionCode_in_brand_prop'],
      dtype='object', length=184)
(150000, 180)
(50000, 180)


In [23]:
train = pd.concat([X_data, Y_data], axis=1, ignore_index=False)
train.to_csv('../data/training/train_data_180.csv', index=False)
X_test.to_csv('../data/training/test_data_180.csv', index=False)